<a href="https://colab.research.google.com/github/wonyoung-ji/NLP/blob/master/5%EC%9D%BC%EC%B0%A8/200904_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_%EB%82%98%EC%9D%B4%EB%B8%8C%EB%B2%A0%EC%9D%B4%EC%A7%80%EC%96%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# 직접구현

In [62]:
import pandas as pd
csv = pd.read_csv('/content/gdrive/My Drive/NLP/naivebayes_example.csv')
csv
#1이 긍정, 0이 부정

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [76]:
from collections import defaultdict

mails = [ csv.iloc[i,0] for i in range(len(csv))]
mail_ls= [ mail.split(' ') for mail in mails ]

word2id = defaultdict(lambda:len(word2id))

for mail in mail_ls:
  for token in mail:
    word2id[token]

print(word2id)

defaultdict(<function <lambda> at 0x7f69fc830f28>, {'i': 0, 'love': 1, 'you': 2, 'happy': 3, 'weekend': 4, 'bore': 5, 'work': 6, 'job': 7, 'hate': 8, 'together': 9})


In [118]:
import numpy as np

#긍정문장만 받기
positive_ls = [] 
for i in range(len(csv)):
  if csv.iloc[i,1] == 1:
    positive_ls.append(csv.iloc[i,0])

#긍정문장 토큰화
positive_ls_copy = [words.split(' ') for words in positive_ls]

#긍정문장 토큰의 등장횟수세기
positive = np.zeros((len(positive_ls),len(word2id)))

for i, words in enumerate(positive_ls_copy):
  for word in words:
    positive[i, word2id[word]] += 1
print(positive)
print('\n\n')

#더하기
real_positive = np.zeros(len(word2id))
for i in range(len(word2id)):
  real_positive[i] = np.count_nonzero(positive[:,i])
real_positive

[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]]





array([1., 2., 1., 2., 1., 0., 0., 0., 0., 1.])

In [119]:
#부정문장만 받기
negative_ls = [] 
for i in range(len(csv)):
  if csv.iloc[i,1] == 0:
    negative_ls.append(csv.iloc[i,0])

#긍정문장 토큰화
negative_ls_copy = [words.split(' ') for words in negative_ls]

#긍정문장 토큰의 등장횟수세기
negative = np.zeros((len(negative_ls),len(word2id)))

for i, words in enumerate(negative_ls_copy):
  for word in words:
    negative[i, word2id[word]] += 1
print(negative)
print('\n\n')

#더하기
real_negative = np.zeros(len(word2id))
for i in range(len(word2id)):
  real_negative[i] = np.count_nonzero(negative[:,i])
real_negative

[[0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]]





array([1., 0., 1., 0., 1., 2., 1., 1., 1., 0.])

In [100]:
#P(w|positive)구하기
p_w_positive=[]

for i in range(len(word2id)):
  p_w_positive.append((df.iloc[0,i] + 1) / (sum(real_positive) + len(word2id))) #라플라스 스무딩 적용

p_w_positive

[0.1111111111111111,
 0.16666666666666666,
 0.1111111111111111,
 0.16666666666666666,
 0.1111111111111111,
 0.05555555555555555,
 0.05555555555555555,
 0.05555555555555555,
 0.05555555555555555,
 0.1111111111111111]

In [102]:
#P(w|negative)구하기
p_w_negative=[]

for i in range(len(word2id)):
  p_w_negative.append((df.iloc[1,i] + 1) / (sum(real_negative) + len(word2id))) #라플라스 스무딩 적용

p_w_negative

[0.1111111111111111,
 0.05555555555555555,
 0.1111111111111111,
 0.05555555555555555,
 0.1111111111111111,
 0.16666666666666666,
 0.1111111111111111,
 0.1111111111111111,
 0.1111111111111111,
 0.05555555555555555]

In [79]:
sorted_vocab = sorted((values,keys) for keys, values in word2id.items())
vocab = [v[1] for v in sorted_vocab]
vocab

['i',
 'love',
 'you',
 'happy',
 'weekend',
 'bore',
 'work',
 'job',
 'hate',
 'together']

In [105]:
df=pd.DataFrame([real_positive, real_negative,p_w_positive,p_w_negative], columns=vocab)
df.index=['positive','negative','p_w_positive','p_w_negative']
df

,i,love,you,happy,weekend,bore,work,job,hate,together
positive,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
negative,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000
p_w_positive,0.111111,0.166667,0.111111,0.166667,0.111111,0.055556,0.055556,0.055556,0.055556,0.111111
p_w_negative,0.111111,0.055556,0.111111,0.055556,0.111111,0.166667,0.111111,0.111111,0.111111,0.055556


In [126]:
import math

h = 'happy'
w = 'weekend'
p = 'p_w_positive'
n = 'p_w_negative'

#P(positive|happy,weekend) 구하기
분자=math.exp(math.log(df.loc[p,h]*df.loc[p,w]*0.5))
분모1=분자
분모2=math.exp(math.log(df.loc[n,h]*df.loc[n,w]*0.5))
print(분자/(분모1+분모2))


#P(negative|happy,weekend) 구하기
분자=math.exp(math.log(df.loc[n,h]*df.loc[n,w]*0.5))
분모1=분자
분모2=math.exp(math.log(df.loc[p,h]*df.loc[p,w]*0.5))
print(분자/(분모1+분모2))


0.7499999999999999
0.2500000000000001


# sklearn 활용하여 구현

In [127]:
X_train = list(csv['mail'])
Y_train = list(csv['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [142]:
from sklearn.feature_extraction.text import CountVectorizer     #빈도수세기
from sklearn.feature_extraction.text import TfidfTransformer    #Tf-Idf 구하기
from sklearn.naive_bayes import MultinomialNB                   #나이브베이지안 구하기

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts)
clf = MultinomialNB().fit(X_train_counts, Y_train)      #fit(a,b):a를 b에 매칭

  (0, 4)	1
  (0, 8)	1
  (1, 4)	1
  (1, 1)	1
  (1, 6)	1
  (2, 0)	1
  (2, 7)	1
  (2, 3)	1
  (3, 8)	1
  (3, 2)	1
  (4, 6)	1
  (4, 0)	1
  (5, 1)	1
  (5, 5)	1


In [137]:
#예측
print(clf.predict(count_vect.transform(['happy weekend'])))
# 확률
print(clf.predict_proba(count_vect.transform(['happy weekend'])))

[1]
[[0.25 0.75]]
